In [3]:
from tokenizers import Tokenizer
from tokenizers import trainers
from tokenizers.normalizers import StripAccents, Lowercase, Sequence
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer, UnigramTrainer
from tokenizers.models import BPE, Unigram

from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
import tokenizers

import re
from hazm import *

In [1]:
! pip install tokenizers scikit-learn


  Using cached tokenizers-0.21.0.tar.gz (343 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (2.1 kB)
  Using cached scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (2.2 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached filelock-3.12.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached fsspec-2023.1.0-py3-none-any.whl.metadata (5.5 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached PyYAML-6.0.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB

In [2]:
! pip install hazm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 24.1 MB/s eta 0:00:00 MB/s eta 0:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 27.2 MB/s eta 0:00:0031m31.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=2cc5842076959ce0cacfd47535b689f4c2b4498b5e182847dfaf1561371524ed
  Stored in directory: /home/jopi-lima/.cache/pip/wheels/65/3c/c7/44672c5062c16d05760b1eaddbf611d2f6a4b715c6d6777418
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-

In [3]:
! pip install tokenizers scikit-learn
! pip install hazm



  Using cached tokenizers-0.21.0.tar.gz (343 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      Checking for Rust toolchain....
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [7]:
!wget https://downloads.wortschatz-leipzig.de/corpora/fas_news_2020_100K.tar.gz
!tar --gunzip --extract --verbose --file=fas_news_2020_100K.tar.gz
persian_text_path = "content/fas_news_2020_100K/fas_news_2020_100K-sentences.txt"

--2024-12-05 11:29:46--  https://downloads.wortschatz-leipzig.de/corpora/fas_news_2020_100K.tar.gz
Resolving downloads.wortschatz-leipzig.de (downloads.wortschatz-leipzig.de)... 139.18.2.68
Connecting to downloads.wortschatz-leipzig.de (downloads.wortschatz-leipzig.de)|139.18.2.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31227186 (30M) [application/x-gzip]
Saving to: ‘fas_news_2020_100K.tar.gz.1’

fas_news_2020_100K. 100%[===================>]  29,78M  21,6MB/s    in 1,4s    

2024-12-05 11:29:47 (21,6 MB/s) - ‘fas_news_2020_100K.tar.gz.1’ saved [31227186/31227186]

fas_news_2020_100K/
fas_news_2020_100K/fas_news_2020_100K-inv_w.txt
fas_news_2020_100K/fas_news_2020_100K-sources.txt
fas_news_2020_100K/fas_news_2020_100K-co_n.txt
fas_news_2020_100K/fas_news_2020_100K-import.sql
fas_news_2020_100K/fas_news_2020_100K-sentences.txt
fas_news_2020_100K/fas_news_2020_100K-co_s.txt
fas_news_2020_100K/fas_news_2020_100K-words.txt
fas_news_2020_100K/fas_news_2020_

In [8]:
# Punctuations and Separators
punc = '''()-[]{};،:'"\\, <>./?@#$%^&*_~.'''
seperator = ['\xad', '\u200e', '\u200f', '\u200d', '\u200c', '\n']

# Hazm Normalizer and Stemmer
hazm_normalizer = Normalizer()
hazm_stemmer = Stemmer()

# Read corpus
with open(persian_text_path, "r") as f:
    sentences = f.readlines()

def preprocess_text_with_hazm(text):
    # Normalize using Hazm e.g. اصلاح نويسه ها -> اصلاح نویسه‌ها
    text = hazm_normalizer.normalize(text)

    # Remove unwanted separators
    for sep in seperator:
        text = text.replace(sep, " ")

    # Remove all punctuation
    text = re.sub(r'[^ا-ی0-9\s]+', '', text)

    # Tokenize using Hazm for more accurate Persian tokenization
    tokens = word_tokenize(text)

    # Add <s> and </s> tags to the sentence
    # tokens = ['<s>'] + tokens + ['</s>']

    # Stemming using hazm e.g  کتاب‌ها -> کتاب‌
    # tokens = [hazm_stemmer.stem(token) for token in tokens]

    return tokens

# Preprocess all sentences
cleaned_sentences = [preprocess_text_with_hazm(sentence) for sentence in sentences]

# Split into training, validation, and testing datasets (80%, 10%, 10%)
train_corpus, temp_corpus = train_test_split(cleaned_sentences, test_size=0.2, random_state=42)
val_corpus, test_corpus = train_test_split(temp_corpus, test_size=0.5, random_state=42)

# Example
print("Training Data Example:", train_corpus[0])
print("Validation Data Example:", val_corpus[0])
print("Test Data Example:", test_corpus[0])

Training Data Example: ['گفتم', 'نه', 'حاجی', 'شما', 'هم', 'بروید', 'دیگر', 'کسی', 'نیست']
Validation Data Example: ['در', 'این', 'خصوص', 'مدیرکل', 'دفتر', 'پیشگیری', 'از', 'قاچاق', 'کالا', 'و', 'ارز', 'در', 'واکنش', 'به', 'اعلام', 'مار', 'قاچاق', 'سیگار', 'در', 'کشور', 'توسط', 'برخی', 'دستگاه', 'ها', 'گفت', 'اعلام', 'رسمی', 'مار', 'قاچاق', 'سیگار', 'در', 'کشور', 'بر', 'عهده', 'ستاد', 'مبارزه', 'با', 'قاچاق', 'کالا', 'و', 'ارز', 'است']
Test Data Example: ['نشان', 'به', 'ن', 'نشان', 'که', 'در', 'حدود', 'کمتر', 'از', 'سال', 'خدمتش', 'همه', 'کیفها', 'پولها', 'چکهای', 'حامل', 'و', 'گوشیهای', 'همراهی', 'که', 'پیدا', 'کرده', 'با', 'پیجویی', 'تمام', 'به', 'صاحبانش', 'برگردانده', 'است']


In [9]:
from nltk import ngrams
from collections import Counter

def ngram_counts(corpus, n, i):
    ngram_list = []

    stopwords = ['به', 'از', 'را', 'و', 'برای', 'این', 'که', 'با', 'در', 'چون', 'اگر', 'ها', 'نه', 'اینکه', 'یا', 'هم', 'تا', 'که', 'آن', 'باید', 'شده', 'چرا', 'همچنین', 'کردن', 'شد', 'می']
    filtered_corpus = [
        [word for word in sentence if word not in stopwords]
        for sentence in corpus
    ]

    for sentence in filtered_corpus:
        sentence_ngrams = list(ngrams(sentence, n))
        ngram_list.extend(sentence_ngrams)

    ngram_counts = Counter(ngram_list)

    most_common_ngrams = ngram_counts.most_common(i)

    print(f"\nMost common {n}-grams:")
    for ngram, count in most_common_ngrams:
        print(f"{ngram}: {count}")

ngram_counts(train_corpus, 3, 10)


Most common 3-grams:
('گزارش', 'همشهری', 'نلاین'): 1115
('همشهری', 'نلاین', 'نقل'): 770
('وی', 'ادامه', 'داد'): 510
('دانشگاه', 'علوم', 'پزشکی'): 397
('مجلس', 'شورای', 'اسلامی'): 384
('جمهوری', 'اسلامی', 'ایران'): 284
('شیوع', 'ویروس', 'کرونا'): 271
('میلیون', 'هزار', 'تومان'): 214
('جان', 'خود', 'دست'): 172
('خبر', 'داد', 'گفت'): 165


In [10]:
ngram_counts(train_corpus, 2, 20)


Most common 2-grams:
('ادامه', 'داد'): 1317
('همشهری', 'نلاین'): 1180
('ویروس', 'کرونا'): 1177
('گزارش', 'همشهری'): 1131
('بر', 'اساس'): 1075
('اعلام', 'کرد'): 969
('وی', 'افزود'): 955
('خبر', 'داد'): 930
('وجود', 'دارد'): 836
('تصریح', 'کرد'): 779
('نلاین', 'نقل'): 771
('هزار', 'نفر'): 690
('ممکن', 'است'): 675
('تاکید', 'کرد'): 670
('بیان', 'کرد'): 666
('نشان', 'دهد'): 648
('وی', 'ادامه'): 642
('هزار', 'تومان'): 628
('اظهار', 'کرد'): 620
('حال', 'حاضر'): 548


In [11]:
from nltk import word_tokenize
from nltk.lm import MLE
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.lm.preprocessing import padded_everygram_pipeline

# Set the n-gram size
n = 3

# Check pre-processing on a subset:

#This preprocessing step helps models capture context effectively, especially at sentence boundaries.

ngram_data, padded = padded_everygram_pipeline(n, train_corpus[0:10])

# What is the effect of padding?

# Padding adds special tokens (start symbol <s>) and (end symbol </s>) to the text to signify sentence boundaries.
# This ensures that n-grams near the edges of a sentence (e.g., start or end) still form complete n-grams
# by including these tokens. For example, in trigrams, first_word and last_word are generated.

print("PADDING:")
print(list(padded))

# Which ngrams do we get?

# Unigrams: Individual words or tokens, e.g., ('<s>',), ('volume',).
# Bigrams: Pairs of tokens, e.g., ('<s>', '<s>'), ('<s>', 'volume').
# Trigrams: Groups of three tokens, e.g., ('<s>', '<s>', 'volume'), ('<s>', 'volume', 'i').
# Padding ensures valid n-grams even at sentence start/end.

print("\n\nNGRAMS:")
for ngrams in ngram_data:
    print(list(ngrams))
    print()

PADDING:
['<s>', '<s>', 'گفتم', 'نه', 'حاجی', 'شما', 'هم', 'بروید', 'دیگر', 'کسی', 'نیست', '</s>', '</s>', '<s>', '<s>', 'در', 'این', 'غربال', 'گری', 'ها', 'کیس', 'های', 'مشکوکی', 'با', 'علامت', 'تب', 'داشتیم', 'که', 'خدا', 'را', 'شکر', 'ورزشکاری', 'نبود', 'که', 'با', 'کرونا', 'درگیر', 'باشد', 'اما', 'در', 'بین', 'همکاران', 'بودند', 'کسانی', 'که', 'مبتلا', 'شدند', '</s>', '</s>', '<s>', '<s>', 'حضرت', 'رسول', 'در', 'حلقه', 'اصحاب', 'نشسته_بودند', '</s>', '</s>', '<s>', '<s>', 'این', 'اتفاق', 'باعث', 'شد', 'تا', 'نقش', 'ایالات', 'متحده', 'به', 'عنوان', 'جایگزین', 'طالبان', 'در', 'افغانستان', 'حذف', 'و', 'جای', 'ن', 'را', 'دولتی', 'بگیرد', 'که', 'بومی', 'و', 'از', 'میان', 'سیاستمداران', 'افغان', 'انتخاب', 'شده_است', '</s>', '</s>', '<s>', '<s>', 'وی', 'اضافه', 'کرد', 'همه', 'بخشهای', 'سازمان', 'در', 'مقابله', 'با', 'کرونا', 'و', 'حفظ', 'رامش', 'مردم', 'از', 'هیچ', 'چیز', 'دریغ', 'نمی', 'کنند', 'و', 'تلاش', 'دارند', 'تا', 'خرین', 'لحظه', 'کار', 'کنند', 'و', 'حتی', 'از', 'جان', 'خود', 'هم'

In [12]:
from nltk.lm import MLE

n=3

# Train data is an iterator over the pre-processed input
train_data, padded_sents = padded_everygram_pipeline(n, train_corpus)

model = MLE(n)
model.fit(train_data, padded_sents)

In [13]:
# Let's have a look at the vocabulary
print(model.vocab)
print()
print(model.vocab.counts.most_common(50))
print()
print(model.vocab.counts.most_common()[-50:])

# You can see that the text is much cleaner than the corpora that were crawled from the web

<Vocabulary with cutoff=1 unk_label='<UNK>' and 53147 items>

[('<s>', 160000), ('</s>', 160000), ('و', 80726), ('در', 68527), ('به', 57193), ('از', 46674), ('این', 36084), ('که', 35698), ('می', 30555), ('را', 28998), ('با', 27376), ('است', 19381), ('برای', 13186), ('کرد', 10405), ('های', 9613), ('شود', 9205), ('یک', 8464), ('هم', 8270), ('ن', 7968), ('تا', 7281), ('گفت', 7094), ('شد', 7089), ('خود', 6456), ('بر', 6087), ('کند', 5926), ('وی', 5506), ('ایران', 5451), ('سال', 5366), ('ما', 5316), ('کشور', 5088), ('ها', 5024), ('نیز', 4921), ('باید', 4874), ('اما', 4792), ('دارد', 4758), ('کنند', 4755), ('کرونا', 4622), ('بود', 4543), ('او', 4449), ('یا', 4290), ('شده', 4133), ('مردم', 4115), ('داد', 4090), ('استان', 3984), ('قرار', 3791), ('روز', 3528), ('اینکه', 3433), ('نها', 3431), ('نمی', 3360), ('دیگر', 3355)]

[('گوادالاخارا', 1), ('جالیسکو', 1), ('خواسته_ای', 1), ('ورشوه', 1), ('چکارمیکنن', 1), ('روچه', 1), ('روو', 1), ('مادگیاش', 1), ('بوردو', 1), ('نینوی', 1), ('سرطانزادیی', 1),

In [ ]:
print(model.counts)
#('دانشگاه', 'علوم', 'پزشکی'): 397
# counts for unigrams:
print(model.counts['پزشکی']) # i.e. Count('not')
# count for bigrams
print(model.counts[['علوم']]['پزشکی']) # i.e. Count('not'|'was')
# count for trigrams
print(model.counts[['دانشگاه', 'علوم']]['پزشکی']) # i.e. Count('not'|'emma was')

<NgramCounter with 3 ngram orders and 6377730 ngrams>
1033
475
397


In [ ]:
all_tokens = [tok for sent in train_corpus for tok in sent]
num_tokens = len(all_tokens)
num_sentences = len(train_corpus)

model_score = model.score('دانشگاه')
probability = model.counts['دانشگاه']/num_tokens


print("\nProbability of the word 'دانشگاه'")
print("{:.5f}".format(model_score))
print("{:.5f}".format(probability))

print("\nAdjust for padding tokens")
all_padding_tokens = num_sentences * (n-1) * 2
print(num_tokens, all_padding_tokens)

adjusted_probability = model.counts['دانشگاه']/(num_tokens + all_padding_tokens)
print("{:.5f}".format(adjusted_probability))

print("\nProbabilities padding tokens")
print("{:.5f}".format(model.score('<s>')))
print("{:.5f}".format(model.score('</s>')))


Probability of the word 'دانشگاه'
0.00059
0.00069

Adjust for padding tokens
1885910 320000
0.00059

Probabilities padding tokens
0.07253
0.07253


In [ ]:
# We can also calculate the frequency for higher n-grams.
# bigram
print(model.score('پزشکی', ['علوم']))  # P('not'|'is')
# trigram
print(model.score('پزشکی', ['دانشگاه', 'علوم']))  # P('not'|'emma is')
print()

0.7078986587183308
0.9925



In [ ]:
# To avoid underflow when working with many small score values, we usually work with log probabilities instead.
# This can be done with the `logscore` method.
#('هزار', 'نفر'): 690
print(model.score('نفر', ['هزار']))
print(model.logscore('نفر', ['هزار']))

0.10015220700152207
-3.3197338814744506


In [ ]:
# The vocabulary helps us handle words that have not occurred during training.
# If we lookup the vocab on unseen sentences not from the training data,
# it automatically replace words not in the vocabulary with `<UNK>`.
print(model.vocab.lookup('در دانشگاه علوم پزشکی موژان قدم می زند'.split()))

('در', 'دانشگاه', 'علوم', 'پزشکی', '<UNK>', 'قدم', 'می', 'زند')


In [ ]:
# Items that are not seen during training are mapped to the vocabulary's "unknown label" token.  This is "<UNK>" by default.
print(model.score("<UNK>") == model.score("موژان"))

# The MLE model does not apply any smoothing, so the probability for UNK is 0
print(model.score("<UNK>"),model.logscore("<UNK>") )

# As a consequence, the probability for a phrase containing an unknown word is also 0.
print(model.score('موژان', ['علوم', 'پزشکی']), model.logscore('موژان', ['علوم', 'پزشکی']))

True
0.0 -inf
0.0 -inf


In [ ]:
from nltk.lm import Laplace
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, train_corpus)
smoothed_model_small =  Laplace(n)
smoothed_model_small.fit(train_data, padded_sents)
print(smoothed_model_small.score('علوم'))
print(smoothed_model_small.score('پزشکی'))
print(smoothed_model_small.score('موژان', ['علوم', 'پزشکی']))
print()
print(smoothed_model_small.logscore('علوم'))
print(smoothed_model_small.logscore('پزشکی'))
print(smoothed_model_small.logscore('موژان', ['علوم', 'پزشکی']))

0.00029746925376384926
0.0004577131077259228
1.8649061952183804e-05

-11.71497181948087
-11.093268771944695
-15.710537409691842


In [ ]:
smoothed_model_small.generate(text_seed=["در", "دانشگاه", "علوم", "پزشکی"], num_words=15, random_seed=1)

['اقدام',
 'به',
 'مثلا',
 'ابطال',
 'واحدهای',
 'صندوق',
 'نماید',
 'در',
 'نتیجه',
 'استرس',
 'اکسیداتیو',
 'جلوگیری',
 'می',
 'شود',
 '</s>']

In [ ]:
test_data, _ = padded_everygram_pipeline(n, val_corpus)

perplexity = []
for test in test_data:
  perplexity.append(model.perplexity(test))

In [ ]:
values = []
#TODO
for i in range(len(perplexity)):
  if not np.isinf(perplexity[i]):
    values.append(i)

In [ ]:
vali1d_perplexity = [perplexity[i] for i in values]
idx = np.argpartition(valid_perplexity, 10)
# vali1d_perplexity[idx[0]]

In [ ]:
print(f"length of the whole test data: {len(perplexity)}")
print(f"length of the filtered test data: {len(vali1d_perplexity)}")

length of the whole test data: 10000
length of the filtered test data: 81


In [ ]:
for i in values:
  print("({0}):{1}".format(val_corpus[i], perplexity[i]))

(['به', 'فاصله', 'کمی', 'از', 'این', 'جنجال', 'ها', 'خبر', 'بازداشت', 'تبریزیان', 'هم', 'رسانهای', 'شد', 'و', 'کانال', 'تلگرامی', 'دولت', 'بهار', 'منتسب', 'به', 'هواداران', 'محمود', 'احمدینژاد', 'ریسجمهور', 'سابق', 'ایران', 'از', 'بازداشت', 'تبریزیان', 'خبر', 'داد']):92.6692647165229
(['برابر', 'میانگین', 'جهانی', 'است']):41.24773731012237
(['براساس', 'اعلام', 'بانک', 'مرکزی']):52.680438408255725
(['متاسفانه', 'در', 'طول', 'ساعت', 'گذشته', 'بیمار', 'کووید', 'جان', 'خود', 'را', 'از', 'دست', 'دادند']):29.39017024389159
(['و', 'بر', 'اساس', 'مصوبه', 'مورخ']):46.12720416408969
(['درصد', 'بوده_است']):22.698559210195157
(['درصد', 'کاهش', 'پیدا', 'کرده_است']):27.649245631191263
(['طی', 'ساعت', 'گذشته', 'این', 'کشور', 'مورد', 'ابتلای', 'جدید', 'به', 'کرونا', 'را', 'شناسایی', 'کرده_است']):58.60122081896455
(['به', 'گزارش', 'همشهرینلاین', 'دکتر', 'سیما', 'سادات', 'لاری', 'سخنگوی', 'وزارت', 'بهداشت', 'گفت', 'از', 'دیروز', 'تا', 'امروز', 'بان', 'و', 'بر', 'اساس', 'معیارهای', 'قطعی', 'تشخیصی', 'هزا

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00


In [ ]:
import tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
with open(persian_text_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 12853517
1	۰۰۰ پرس غذا برگزار خواهد گردید که با توجه به شرایط کرونایی کشور همه این غذاها در سطح شهر و مناطق 


In [ ]:
with open(persian_text_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

# Remove newlines and duplicated whitespaces
raw_text = raw_text.replace("\n", " ")
raw_text = raw_text.replace("  ", " ")
enc_text = tokenizer.encode(raw_text)

In [ ]:
print(raw_text[0:100])
print()
print(tokenizer.decode(enc_text[0:50]))
print()
tokens = [tokenizer.decode([tok]) for tok in enc_text[0:50]]
print(enc_text[0:50])
print()
print(tokens)

1	۰۰۰ پرس غذا برگزار خواهد گردید که با توجه به شرایط کرونایی کشور همه این غذاها در سطح شهر و مناطق م

1	۰۰۰ پرس غذا برگزار خواهد گردید که با تو

[16, 197, 151, 108, 151, 108, 151, 108, 18923, 122, 26897, 45692, 17550, 118, 148, 108, 12919, 17550, 101, 26897, 150, 107, 148, 110, 12919, 26897, 17550, 106, 30335, 12919, 29519, 38843, 220, 150, 107, 26897, 38843, 151, 234, 38843, 220, 150, 102, 29519, 17550, 101, 12919, 17550, 103, 30335]

['1', '\t', '�', '�', '�', '�', '�', '�', ' �', '�', 'ر', 'س', ' �', '�', '�', '�', 'ا', ' �', '�', 'ر', '�', '�', '�', '�', 'ا', 'ر', ' �', '�', 'و', 'ا', 'ه', 'د', ' ', '�', '�', 'ر', 'د', '�', '�', 'د', ' ', '�', '�', 'ه', ' �', '�', 'ا', ' �', '�', 'و']


In [ ]:
enc_sample = train_corpus[7:99]

context_length = 4
for i in range(1, context_length+1):
    context = enc_sample[:i]
    target = enc_sample[i]

    print(context, "-->", target)
    # print(tokenizer.decode(context), "-->",tokenizer.decode([target]))

[['ارتفاع', 'پست', 'ابراهیم', 'حاتمیکیا', 'هم', 'پسزمینه', 'جنگ', 'دارد', 'و', 'پیامدهای', 'جنگ', 'را', 'بر', 'زندگی', 'مردم', 'خوزستان', 'روایت', 'می', 'کند', 'و', 'سرانجام', 'بمب', 'یک', 'عاشقانه', 'پیمان', 'معادی', 'که', 'داستان', 'ن', 'در', 'دوران', 'موشکباران', 'شهرها', 'روایت', 'می', 'شود']] --> ['وی', 'ادامه', 'داد', 'این', 'مسیر', 'به', 'علت', 'پایین', 'مدن', 'چند', 'قطعه', 'سنگ', 'بزرگ', 'بر', 'اثر', 'برخورد', 'صاعقه', 'با', 'کوه', 'مسدود', 'شده_است', 'و', 'در', 'سایه', 'تلاش', 'راهدارای', 'و', 'جابهجایی', 'سنگهای', 'کوچک', 'این', 'راه', 'روستایی', 'به', 'صورت', 'موقت', 'و', 'کنارگذر', 'بازگشایی', 'شده_است']
[['ارتفاع', 'پست', 'ابراهیم', 'حاتمیکیا', 'هم', 'پسزمینه', 'جنگ', 'دارد', 'و', 'پیامدهای', 'جنگ', 'را', 'بر', 'زندگی', 'مردم', 'خوزستان', 'روایت', 'می', 'کند', 'و', 'سرانجام', 'بمب', 'یک', 'عاشقانه', 'پیمان', 'معادی', 'که', 'داستان', 'ن', 'در', 'دوران', 'موشکباران', 'شهرها', 'روایت', 'می', 'شود'], ['وی', 'ادامه', 'داد', 'این', 'مسیر', 'به', 'علت', 'پایین', 'مدن', 'چند', 'ق

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, context_length):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - context_length):
            input_sequence = token_ids[i:i + context_length]

            #shift to the right
            target_sequence = token_ids[i + 1: i + context_length + 1]

            # input and output are represented as tensors
            self.input_ids.append(torch.tensor(input_sequence))
            self.target_ids.append(torch.tensor(target_sequence))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader(txt, batch_size=8, context_length=4, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDataset(txt, tokenizer, context_length)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
# Set a manual seed for reproducibility of shuffling and weight initialization
torch.manual_seed(0)
dataloader = create_dataloader(raw_text, batch_size=8, context_length=4, shuffle=False)

In [ ]:
print(len(dataloader))
data_iter = iter(dataloader)

first_batch = next(data_iter)
print("1:", first_batch)